## Лабораторная работа 4: topic modeling

В данной лабораторной работе мы попытаемся обучить LDA-модель topic-моделингу на двух принципиально различных корпусах. 

В первой части вы познакомитесь с новыми возможностями библиотеки gensim, а также с возможностями парсинга в языке Python. Во второй части вам предстоит самостоятельно обучить LDA-модель и оценить качество её работы.

### Часть 1: topic modeling уровня /b/

Краеугольным камнем в машинном обучений в целом, и в NLP в частности, является выбор датасетов. Доселе мы использовали только стандартные, многократно обкатанные датасеты, но сегодня попробуем собрать свой. Практика работы с сырыми, необработанными данными весьма полезно! Заодно изучим возможности парсеров в Питоне.

Давайте напишем парсер, собирающий информацию о сообщения с русскомязычного анононимного форума (имиджборды) "Двач" ("Сосач", "Хиккач", если вам угодно). Двач, как и всякая имиджборда разделён на разделы (доски, борды), посвященные различным тематикам -- аниме, видеоигры, литература, религия... Каждая доска состоит из тем (тредов, топиков), которые создаются анонимными (при их желании) пользователями. Каждый тред посвящен обсуждению какого-то конкретного вопроса.

У некоторых разделов есть раздел архив, располагается он по адресу https://2ch.hk/(название раздела)/arch/, например для раздела музыка -- https://2ch.hk/mu/arch/. Если у вас есть минимальные навыки в языке html, а также если вы изучили документацию встроенного класса HTMLParser, то вам будет несложно написать два парсера.

Первый парсер (ArchiveParser) парсит HTML-страницу архива доски, вытягивает из неё ссылки на заархивированные треды, и скармливает их второму парсеру.

Второй парсер (ThreadParser) парсит HTML-страницу заархивированного треда, вытягивает из неё сообщения, складывает их вместе и собирает.

In [ ]:
import time
import urllib.request
from html.parser import HTMLParser
from gensim.utils import simple_preprocess

def get_value_by_key(attrs, key):
    for (k, v) in attrs:
        if(k == key):
            return v;
    return None

class ArchiveParser(HTMLParser):
    flag = False
    threads = []
    limit = 200
    def handle_starttag(self, tag, attrs):
        if(self.limit > 0):
            if(tag == 'div'):
                cl = get_value_by_key(attrs, 'class')
                if (cl == 'box-data'):
                    self.flag = True;
            if(self.flag == True and tag == 'a'):
                href = get_value_by_key(attrs, 'href')
                if(len(href)>20):
                    print(href)
                    print(self.limit)
                    thread = parse_thread('https://2ch.hk' + href)
                    if(len(thread) > 10):
                        self.threads.append(thread)
                        self.limit = self.limit - 1
                    thread = []
        

    def handle_endtag(self, tag):
        if(tag == 'div'):
            self.flag = False;

    def handle_data(self, data):
        1+1
        
    def get_threads(self):
        return self.threads
    
    def clean(self):
        self.threads = []
        
parser = ArchiveParser()

def parse_archive(board = '/b/', page_number = 0):
    lines = []
    link = 'https://2ch.hk' + board + 'arch/' + str(page_number) +'.html'
    print(link)
    parser.limit = 100
    url = urllib.request.urlopen(link)
    for line in url.readlines():
        lines.append(line.decode('utf-8'))
    for line in lines:
        parser.feed(line)
    res = parser.get_threads()
    parser.clean()
    return res

In [ ]:
class ThreadParser(HTMLParser):
    flag = False
    message = []
    messages = []
            
    def handle_starttag(self, tag, attrs):
        if(tag == 'blockquote'):
            self.flag = True;
            self.message = []
        

    def handle_endtag(self, tag):
        if(tag == 'blockquote'):
            self.flag = False
            if(self.message != []):
                self.messages.append(self.message)
            self.message = []

    def handle_data(self, data):
        if(self.flag):
            self.message.extend(simple_preprocess(data))
            
    def get_messages(self):
        return self.messages
    
    def clear_messages(self):
        flag = False
        self.message = []
        self.messages = []

t_parser = ThreadParser()

def parse_thread (link):
    url = urllib.request.urlopen(link)
    lines = []
    for line in url.readlines():
        lines.append(line.decode('utf-8', errors='ignore'))
    for line in lines:
        t_parser.feed(line)
    res = t_parser.get_messages()
    t_parser.clear_messages()
    #print(res)
    return res

Весьма много кода, верно? Если не потерялись, могли заметить функцию parse_archive, которая парсит страницу архива по доске и номеру страницы.


$\textbf{Задание.}$
Давайте применим её к каким-нибудь доскам. Выберите две доски двача, имеющие архив и скачайте архивы функцией parse_archive.

In [ ]:
boards = ['] #TODO: напишите название досок в формате /'доска'/, например /mu/ для Музыки
threads_by_topic = [parse_archive(board=board) for board in boards]

Разделим наши данны на тренировочые и тестовые. Пусть каждый десятый тред попадает в тест-сет.

In [ ]:
data = []
test = []

it = 0
for topic in threads_by_topic:
    for thread in topic:
        full = []
        for post in thread:
            full.extend(post)
        it = it + 1
        if(it % 10 == 0):
            test.append(full)
        else:
            data.append(full)

            

$\textbf{Задание.}$
В русском языке есть множество слов (частицы, междометия, всё что вы хотите), которые никак не отображают смысл слов и являются вспомогательными. Чтобы ваша модель работала лучше -- добавьте стоп-слова в список RUSSIAN_STOP_WORDS или в строку st_str. Эти слова отфильтруются из датасета перед тем, как модель начнет обучаться на датасете.

In [ ]:
from gensim.utils import simple_preprocess
from gensim import corpora

RUSSIAN_STOP_WORDS = ['не', 'это', 'что','чем','это','как','https','нет','op','он','же','так','но','да','нет','или','и', 'на', "то", "бы", "все", "ты", "если", "по", "за", "там", "ну", "уже", "от", "есть","был", "даже", "было", "www", "com", "youtube", "из", "будет", "mp", "они", "только", "его", "она", "вот", 'просто', 'watch', 'кто', 'для', 'когда', 'тут', 'мне', 'где', 'мы', 'какой', 'может', 'меня', 'до', 'про', 'http', 'раз', 'почему', 'тебя', 'ещё', 'их', 'сейчас', 'тоже', 'во', 'чтобы', 'этого','без', 'него','вы','такой', 'можно', 'надо', 'нахуй', 'ли', 'потом', 'тред', 'больше', 'лучше', 'хуй', 'сам', 'после', 'со', 'лол', 'быть', 'нужно', 'этом', 'блять', 'бля', 'того', 'ничего', 'потому', 'нибудь', 'этот', 'под', 'через', 'ни', 'себе', 'ему', 'при', 'какие', 'пиздец', 'теперь', 'хоть', 'говно', 'тогда', 'блядь', 'кстати', 'че', 'себя', 'конечно', 'типа', 'много', 'том', 'нихуя', 'куда', 'всегда', 'нас', 'тот', 'ведь', 'эти', 'них', 'сука', 'пока', 'более', 'чего', 'html', 'были', 'всех', 'была', 'например', 'тем', 'ru', 'зачем', 'либо', 'вроде', 'всего', 'вопрос', 'php', 'против', 'здесь', 'ее', 'значит', 'совсем', 'сколько', 'им', 'org', 'именно', 'эту',]
st_str = "которых которые твой которой которого сих ком свой твоя этими слишком нами всему будь саму чаще ваше сами наш затем еще самих наши ту каждое мочь весь этим наша своих оба который зато те этих вся ваш такая теми ею которая нередко каждая также чему собой самими нем вами ими откуда такие тому та очень сама нему алло оно этому кому тобой таки твоё каждые твои мой нею самим ваши ваша кем мои однако сразу свое ними всё неё тех хотя всем тобою тебе одной другие этао само эта буду самой моё своей такое всею будут своего кого свои мог нам особенно её самому наше кроме вообще вон мною никто это"
RUSSIAN_STOP_WORDS.extend(st_str.split(' '))

data = [list(filter(lambda word: not word in RUSSIAN_STOP_WORDS, piece)) for piece in data]
id2word = corpora.Dictionary(data)

Создадим словарь и на его основе преобразуем слова в их id.

In [ ]:
id2word = corpora.Dictionary(data)

# Create Corpus
texts = data

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


Обучим LDA-модель, используя библиотеку gensim. Зададим число тем равно числу скачанных досок.

In [ ]:
from gensim.models import LdaModel

model = LdaModel(corpus, id2word=id2word, num_topics=len(threads_by_topic))

Теперь получим топ-10 самых используемых в каждой теме слов.

$\textbf{Задание.}$
Оцените насколько хорошо модель разделила темы.

In [ ]:
for i in range(len(threads_by_topic)):
    print([id2word[id[0]] for id in model.get_topic_terms(topicid = i, topn = 10)])

Теперь прогоним тестовые треды на модели. Тестовый датасет разделен на n равных частей по 20 тредов, i-ая соответствует i-й доске.

In [ ]:
other_corpus = [id2word.doc2bow(text) for text in [list(filter(lambda word: not word in RUSSIAN_STOP_WORDS, piece)) for piece in test]]

vector = [model[unseen_doc] for unseen_doc in other_corpus]
print(vector[0]) #вероятности принадлежности 0-го тестового треда в ту или иную тему

In [ ]:
i = 0

for res in vector:
    max_it = 0
    if(len(res) > 0):
        for it in range(1, len(res)):
            if(res[max_it][1] < res[it][1]):
                max_it = it
        print("Text #" + str(i) + ", topic #" + str(max_it) + str(", prob = " + str(res[max_it][1])))
    i = i + 1

$\textbf{Задание.}$

Оцените результаты работы модели на тест сете. Если модель разделили данные плохо -- объясните, почему?

## Часть 2. А теперь нормальный датасет.

А теперь давайте воспользуемся более стандартным датасетом библиотеки sklreatn -- 20newsgroups, посвященную статьям на различные темы. Выберем 6 -- Атеизм, яблочное железо, автомобили, хоккей, космос, христианство, ближний восток.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism',
 'comp.sys.mac.hardware',
 'rec.autos',
 'rec.sport.hockey',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.mideast']
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories = categories)

$\textbf{Задание}$

Найдите библиотечный или опишите свой список ENGSLISH_STOP_WORDS, убирающий не несущие никакого смысла английские слова.

In [ ]:
from gensim.utils import simple_preprocess
from gensim import corpora

ENGLISH_STOP_WORDS = [] #TODO

print('the' in ENGLISH_STOP_WORDS) 

data = [list(filter(lambda word: not word in ENGLISH_STOP_WORDS, simple_preprocess(piece))) for piece in newsgroups_train.data]

$\textbf{Большое задание 1.}$

Для списка data создайте словарь id2word. Получите преобразованный TermDocumentFrequency список corpust и обучите на нем LDA модель.

In [ ]:
from gensim.models import LdaModel, LsiModel

print(data[0])
id2word = #TODO

# Create Corpus
texts = data

# Term Document Frequency
corpus = #TODO

# View
print(corpus[:1])

model = #TODO

In [ ]:
#Выведем получившийся список тем:
for i in range(len(categories)):
    print([id2word[id[0]] for id in model.get_topic_terms(topicid = i, topn = 10)])

$\textbf{Большое задание 2.}$

В соответствии с тренировочными, обработайте тестовые данные.

Напишите функцию, которая с помощью модели возвращает наиболее вероятный id темы. С помощью F-меры оцените правильность работы модели.

In [ ]:
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories = categories)

#TODO: YOUD CODE